In [46]:
'''
trying other methords to install libraries
'''
try: 
    import scrapy
except:
    !pip install scrapy
import scrapy

from scrapy.crawler import CrawlerProcess

import csv
import logging

In [53]:
'''
Find a methord to write output to csv

Using the pipeline methord, dont understand it yer just trying it out
'''

class CsvWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('restaurant.csv', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = csv.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [54]:
'''
Create the spider

Log :
1. Using the pipeline methord found from google to write output directly to csv 
(will need to study the custome_settings further)
'''

class AlldinerSpider(scrapy.Spider):
    name = 'alldiners'
    allowed_domains = ['eazydiner.com']
    start_urls = ['https://www.eazydiner.com/restaurants?location=delhi-ncr&pax=2']
    custom_settings = {
        'LOG_LEVEL' : logging.WARNING,
        'ITEM_PIPELINES' : {'__main__.CsvWriterPipeline': 1},
        'FEEDS' : {
            'restaurant.csv':{
                'format' : 'csv',
                'overwrite' : True
            }
        }
    }
    

    def parse(self, response):
        
        for item in response.css("div[class='padding-10 radius-4 bg-white restaurant margin-b-10']"):
            
            yield {
                'restaurant_name': item.css("h3[class='grey res_name font-20 bold inline-block']::text").get().strip(),

                'restaurant_location': item.css("h3[class='margin-t-5 res_loc']::text").get().strip(),

                'cost_for_2': item.css("span[class='padding-l-10 grey cost_for_two']::text").get('').strip()[:-7],

                'cusisine': item.css("div[class='grey padding-l-10 res_cuisine']::text").get().strip(),

                'rating': item.css("span.critic::text").get().strip(),

                'restaurant_img': item.css("img[class='radius-4 res_name lazy']::attr(data-src)").get(),

                'restaurant_page': "https://www.eazydiner.com/"+item.css("a[class='btn btn-primary height-40 block bold padding-10 font-14 apxor_click']::attr(href)").get()
            } 

        next_page = response.css('a[rel=next]::attr(href)').get()  
        if next_page is not None:
                yield response.follow(next_page,callback=self.parse)   
                

In [55]:
'''
write a function to run the crawler
'''
def run_spider():
    crawler = CrawlerRunner()
    return crawler.crawl(AlldinerSpider)

In [56]:
final_output = run_spider()

In [57]:
final_output